In [ ]:
from src.tree_parser import *

with open("../tmp/im2latex_formulas.tree", "rb") as lines:
    i = 0
    for tree in iter_tree_lines(lines):
        try:
            if tree:
                list(iter_elements(tree))
                i += 1
        except:
            print(b''.join(tree).decode())
            raise
        
i